In [1]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from graphdatascience import GraphDataScience
from getpass import getpass
import pandas as pd
from langchain_core.runnables import RunnableConfig
import time

/Users/nathansmith/anaconda3/envs/chatbot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Set up connections

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
openai_api_key = getpass()

 ········


In [4]:
neo4j_password = getpass()

 ········


In [5]:
neo4j_uri = "neo4j+s://bc9f751a.databases.neo4j.io"
neo4j_user = "neo4j"
gds = GraphDataScience(neo4j_uri, auth=(neo4j_user, neo4j_password))

# Get document data

In [65]:
doc_info = gds.run_cypher(
    """MATCH (d:Document) 
       WHERE NOT EXISTS {(d)-[:HAS_TOPIC]->()}
       AND d.text is not null
       RETURN d.index AS index, d{.text} AS text""" )

In [7]:
config = RunnableConfig(max_concurrency=4)

In [24]:
chat = ChatOpenAI(temperature=0, model="gpt-4-turbo-preview", openai_api_key=openai_api_key, request_timeout =5000)

In [27]:
gds.run_cypher("""CREATE CONSTRAINT topic_node_key IF NOT EXISTS FOR (n:Topic) REQUIRE n.description IS NODE KEY""")

""


In [39]:
doc_info

,index,text
0,4,"{'text': 'relationshipQuery no Cypher query to project relationships. The query result must contain source and target columns. Optionally, a type column can be specified to represent relationship type. Additional columns are interpreted as properties. configuration yes Additional parameters to configure the Legacy Cypher projection. readConcurrency Integer The number of concurrent threads used for creating the graph. validateRelationships Boolean true'}"
1,5,{'text': 'validateRelationships Boolean true Whether to throw an error if the relationshipQuery returns relationships between nodes not returned by the nodeQuery. parameters Map {} A map of user-defined query parameters that are passed into the node and relationship queries. jobId String Generated internally An ID that can be provided to more easily track the projection’s progress. graphName String The name under which the graph is stored in the catalog. nodeQuery String'}
2,6,"{'text': 'nodeQuery String The Cypher query used to project the nodes in the graph. nodeCount Integer The number of nodes stored in the projected graph. relationshipQuery String The Cypher query used to project the relationships in the graph. relationshipCount Integer The number of relationships stored in the projected graph. projectMillis Integer Milliseconds for projecting the graph. To get information about a stored graph, such as its schema, one can use gds.graph.list. 3. Examples'}"
3,7,{'text': '3. Examples In order to demonstrate the GDS Graph Project capabilities we are going to create a small social network graph in Neo4j. The example graph looks like this: The following Cypher statement will create the example graph in the Neo4j database: 3.1. Simple graph'}
4,8,"{'text': '3.1. Simple graph A simple graph is a graph with only one node label and relationship type, i.e., a monopartite graph. We are going to start with demonstrating how to load a simple graph by projecting only the Person node label and KNOWS relationship type. Project ""persons"" ""MATCH (n:Person) RETURN id(n) AS id"" ""MATCH (n:Person)-[r:KNOWS]→(m:Person) RETURN id(n) AS source, id(m) AS target"" 3.2. Multi-graph A multi-graph is a graph with multiple node labels and relationship types.'}"
...,...,...
15282,6877387f-e924-49a6-9855-e4670337ce4f,"{'text': 'let's just say, Neo4jexample, we wanna explore characters from a particular planet. All right. And that's it. Not a single lineNeo4jcode was writtenNeo4jthis. We just hit Enter and out pops 136 nodes. So from here, you can kindNeo4jjust take a look around. What do we see? So right here, we have Tatooine. We have these various characters. We got Luke Skywalker, obviously. Anakin Skywalker, Darth Vader. And so, you know, you can zoom out again and just start exploring other areas. We got Naboo over here. R2D2. And you can see very clearly, just basedNeo4jthe color code that we assigned eachNeo4jthese categories. It's very responsive. You can drag these nodes around and really manipulate it however you want to. Make these relationship lines longerNeo4jshorter, just so you can clearly see what's happeningNeo4jthe scene. From here, go ead and zoom out. Let's go ead and clear the scene. Neo4jexample, soNeo4jMac, that's gonna be Command and Backspace, to clear the scene. Let's clear our search parameters here. Let's say we want to explore what our Characters IDENTIFIES_AS, so what species they identify as. All right. So we can go ead and do that. Not a single lineNeo4jcode written, again. And boom, here we go. We got a bunchNeo4jnodes and relationship. We have 119 nodes right there. Let's go ead and zoom inNeo4jthisNeo4j. So the biggest clusterNeo4jall is Hans, of course. And theseNeo4jallNeo4jthe various characters in the dataset that identifies as hans. So that's kinda cool. I mean, your mileage may vary, of course. And I did not go into anyNeo4jthe datasets to clean it up whatsoever. JustNeo4jthe purposeNeo4jthis demo, I just wanted to show you all how easy i

# Ask gen AI to find topics in documents

In [29]:
system_message = SystemMessage(
    content="""You are an expert about Neo4j software. 
    Summarize the text of this document into a list of up to fifteen short phrases that describe the topics in the text.
    Try to make the phrases no longer than three or four words.
    The topics should all relate to Neo4j software, Cypher syntax, database administration, graphs, or data science.
    Do not return topics based on people's names, movies, sports, or other topics not directly related to answering technical questions about software.
    If the document is defining Cypher syntax, extract the key function names, Cypher operators, and parameters as phrases.
    If you find a phrase that is a single word used in the Cypher query language, append the words "in Cypher" to the phrase. 
    For example instead of returning the single word "WITH" return "WITH in Cypher" 
    You can provide fewer than fifteen phrases.
    Return the phrases as a pipe separated list.""")

final_prompt = ChatPromptTemplate.from_messages(
    [system_message,
     ("human", "{text}")])

summary_chain = final_prompt | chat

In [53]:
def get_topics(start_id, end_id):
    doc_data = doc_info.iloc[start_id: end_id, :].copy()
    try:
        response = summary_chain.batch(doc_data['text'].to_list(), config=config)
        doc_data.loc[:, 'topics'] = [message.content for message in response]
        topic_df = doc_data['topics'].str.split('|', expand=True).stack()
        topic_df = topic_df.reset_index(level=1, drop=True).to_frame('topic').join(doc_data.drop('topics', axis=1))
        gds.run_cypher("""
            UNWIND $data as row
            MATCH (d:Document {index:row['index']})
            MERGE (t:Topic {description: trim(row['topic'])})
            WITH d, t
            MERGE (d)-[:HAS_TOPIC]->(t)
            RETURN count(*) AS relCount""",
                       {"data": topic_df[['topic', 'index']].to_dict("records")})
    except Exception as e:
        print(e)
        print(doc_data['text'])
        print(doc_data['topics'])

In [66]:
doc_info.shape

(2472, 2)

In [67]:
for i in range(0, int(doc_info.shape[0]/10) + 1):
    get_topics(i*10,(i+1)*10)
    time.sleep(3)
    if (i+1)%5 == 0:
        print(f"Finished row {(i+1)*10}")
    

Finished row 50
Finished row 100
Finished row 150
Finished row 200
Finished row 250
Finished row 300
Finished row 350
Finished row 400
Finished row 450
Finished row 500
Finished row 550
Finished row 600
Finished row 650
Finished row 700
Finished row 750
Finished row 800
Finished row 850
Finished row 900
Finished row 950
Finished row 1000
Finished row 1050
Finished row 1100
Finished row 1150
Finished row 1200
Finished row 1250
Finished row 1300
Finished row 1350
Finished row 1400
Finished row 1450
Finished row 1500
Finished row 1550
Finished row 1600
Finished row 1650
Finished row 1700
Finished row 1750
Finished row 1800
Finished row 1850
Finished row 1900
Finished row 1950
Finished row 2000
Finished row 2050
Finished row 2100
Finished row 2150
Finished row 2200
Finished row 2250
Finished row 2300
Finished row 2350
Finished row 2400
Finished row 2450
